In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [19]:
data = pd.read_csv("train.csv", engine='python', on_bad_lines= 'skip')
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [20]:
data.shape

(20822, 5)

In [21]:
data.isnull().sum()

id           0
title      565
author    1975
text        59
label       22
dtype: int64

In [22]:
###Drop Nan Values
data= data.dropna()

In [23]:
data.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [24]:
data.shape

(18285, 5)

In [25]:
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [26]:
data['label'].value_counts()

label
0                                                   10361
1                                                    7922
 как люди воспринимают своё положение.                  1
 чтобы это была дорога с двусторонним движением.        1
Name: count, dtype: int64

In [27]:
print(data['label'].unique()[:20])  # أول 20 قيمة مختلفة

['1' '0' ' как люди воспринимают своё положение.'
 ' чтобы это была дорога с двусторонним движением.']


In [28]:
# data.drop(data[data['label']==' как люди воспринимают своё положение.'].index, inplace= True)
# data.drop(data[data['label']==' чтобы это была дорога с двусторонним движением.	'].index, inplace= True)

In [29]:
data = data[data['label'] != ' как люди воспринимают своё положение.']
data = data[data['label'] != ' чтобы это была дорога с двусторонним движением.']


In [30]:
data['label'].value_counts()

label
0    10361
1     7922
Name: count, dtype: int64

In [31]:
X = data.drop(columns=['label'])
y = data['label']

In [32]:
import tensorflow as tf
tf.__version__

'2.20.0'

In [33]:
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN, LSTM, Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot

In [34]:
import re
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\IT\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\IT\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [35]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
nltk.download("stopwords")
nltk.download("wordnet")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\IT\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\IT\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [36]:
voc_size = 5000

In [37]:
messages = X.copy()

### Apply Cleaning Processing

In [38]:
ps = PorterStemmer()

In [39]:
stop_words = set(stopwords.words("english"))
stop_words

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 "he'd",
 "he'll",
 "he's",
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 "i'd",
 "i'll",
 "i'm",
 "i've",
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it'd",
 "it'll",
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'on

In [40]:
def cleaning(text):
    text = re.sub(r"<.*?>", "", text)                        #Also Remove HTML tags
    text = text.lower()                                      # Lowercase
    text = re.sub(r'_+', ' ', text)                           # يشيل أي underscores متكررة
    text = re.sub(r"http\S+|www\S+|https\S+", '', text)      # Remove URLs
    text = re.sub(r'\d+', '', text)                          # Remove digits
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)              # Remove all single characters from text
    text = re.sub(r"[^\w\s]", "", text)                      # Remove punctuation
    text = re.sub(r'\s+', ' ', text).strip()                 # Remove extra whitespace
    words = text.split()                                     # Tokenize
    words = [ps.stem(word) for word in words if word not in stop_words]  # Remove stop words
    words = [re.sub(r'(.)\1{2,}', r'\1\1', word) for word in words]  # Limit repeated letters to 2 max

    clean_text = ' '.join(words)                                # Join back into one string
    clean_text = re.sub(r'\b(\w+)( \1\b)+', r'\1', clean_text)  # Remove repeated consecutive words

    return clean_text

In [41]:
messages.drop(columns= ['id'], inplace = True)

In [42]:
messages.head()

,title,author,text
0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [43]:
messages['title_clean'] = messages['title'].apply(cleaning)
# messages = messages["text"].apply(cleaning)
messages[["title", "title_clean"]].sample(7)

,title,title_clean
7437,"Ex-rep: 'If Trump loses, I'm grabbing my musket'",exrep trump lose im grab musket
2321,Time: Investigating Hillary is an Attack on Al...,time investig hillari attack women
8721,VIDEO: Cartel Gunmen Hold Up Border City Curre...,video cartel gunmen hold border citi currenc e...
10007,"On Deck With China’s Last Junk Builders, Maste...",deck china last junk builder master eb craft n...
16399,"2 Reminders to Ignore the ""Trump is Doomed"" Polls",remind ignor trump doom poll
18160,"Russian Fighter Makes ‘Provocative, Erratic’ A...",russian fighter make provoc errat approach wit...
15375,Most College Students Think America Invented S...,colleg student think america invent slaveri pr...


In [44]:
corpus = messages['title_clean']
corpus

0        hous dem aid didnt even see comey letter jason...
1          flynn hillari clinton big woman campu breitbart
2                                     truth might get fire
3                 civilian kill singl us airstrik identifi
4        iranian woman jail fiction unpublish stori wom...
                               ...                        
20817         rapper ti trump poster child white supremaci
20818        nfl playoff schedul matchup odd new york time
20819    maci said receiv takeov approach hudson bay ne...
20820             nato russia hold parallel exercis balkan
20821                                          keep f aliv
Name: title_clean, Length: 18283, dtype: object

## ONE HOT Representation

In [45]:
onehot_repr=[one_hot(words,voc_size)for words in corpus]
onehot_repr

[[2683, 2339, 71, 4766, 4146, 3796, 4718, 4940, 4938, 3780, 2741],
 [3302, 3909, 3093, 551, 3427, 3231, 3773],
 [532, 3569, 4249, 3064],
 [2577, 3605, 1371, 4438, 12, 2537],
 [158, 3427, 2029, 4518, 2154, 4704, 3427, 3071, 4798, 3731],
 [4880,
  602,
  3501,
  2341,
  4080,
  547,
  2675,
  307,
  4604,
  1180,
  3112,
  3715,
  3653,
  1663,
  3773],
 [2014, 4904, 4913, 4361, 1638, 3294, 423, 3714, 4821, 4642, 2257],
 [4451, 848, 530, 1542, 3247, 547, 281, 4821, 4642, 2257],
 [723, 2358, 296, 3218, 2270, 132, 1789, 547, 237],
 [3386, 3463, 4967, 1177, 1535, 2999, 578, 4559],
 [4435, 2745, 4019, 3380, 4838, 1228, 1924, 3546, 2528, 2405, 280],
 [4438, 1653, 3796, 950, 547, 1542],
 [3433, 4721, 2653, 3796, 1942, 3931, 2880, 3682, 281],
 [1343, 660, 3021, 4553, 3153, 2282, 2204, 4821, 4642, 2257],
 [3381, 1805, 4599, 4204, 2525, 4821, 4642, 2257],
 [40, 878, 1629, 383, 1405, 4477, 3594, 4657, 2972, 1856],
 [2723, 2396, 3909],
 [3204, 617, 1498, 3326, 547, 930, 1884, 3773],
 [4968, 3902, 3

In [46]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [47]:
onehot_repr[1]

[3302, 3909, 3093, 551, 3427, 3231, 3773]

## Embedding Representation

In [48]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr, padding='post', maxlen=sent_length)
print(embedded_docs)

[[2683 2339   71 ...    0    0    0]
 [3302 3909 3093 ...    0    0    0]
 [ 532 3569 4249 ...    0    0    0]
 ...
 [3654  251 3988 ...    0    0    0]
 [ 179 1542 2611 ...    0    0    0]
 [2777 3757 4789 ...    0    0    0]]


In [49]:
embedded_docs[1]

array([3302, 3909, 3093,  551, 3427, 3231, 3773,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

## Creating The model

In [50]:
## Creating model
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length=sent_length))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1, activation='sigmoid'))

# نعمل build للموديل
model.build(input_shape=(None, sent_length))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())


c:\ProgramData\anaconda3\envs\myenv\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 20, 40)         │       200,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 200)            │       112,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           201 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 313,001 (1.19 MB)

 Trainable params: 313,001 (1.19 MB)

 Non-trainable params: 0 (0.00 B)

None


In [51]:
len(embedded_docs),y.shape

(18283, (18283,))

In [52]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [53]:
X_final.shape,y_final.shape

((18283, 20), (18283,))

In [54]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [55]:
y_train = y_train.astype(int)
y_test = y_test.astype(int)

### Model Training

In [56]:
### Finally Training
model.fit(X_train, y_train, validation_split=0.2, epochs=10, batch_size=64)

Epoch 1/10
154/154 ━━━━━━━━━━━━━━━━━━━━ 8s 28ms/step - accuracy: 0.8542 - loss: 0.3015 - val_accuracy: 0.9073 - val_loss: 0.2129
Epoch 2/10
154/154 ━━━━━━━━━━━━━━━━━━━━ 4s 25ms/step - accuracy: 0.9510 - loss: 0.1281 - val_accuracy: 0.9147 - val_loss: 0.2088
Epoch 3/10
154/154 ━━━━━━━━━━━━━━━━━━━━ 4s 24ms/step - accuracy: 0.9720 - loss: 0.0812 - val_accuracy: 0.9163 - val_loss: 0.2133
Epoch 4/10
154/154 ━━━━━━━━━━━━━━━━━━━━ 4s 25ms/step - accuracy: 0.9823 - loss: 0.0563 - val_accuracy: 0.9082 - val_loss: 0.2818
Epoch 5/10
154/154 ━━━━━━━━━━━━━━━━━━━━ 5s 31ms/step - accuracy: 0.9863 - loss: 0.0431 - val_accuracy: 0.9102 - val_loss: 0.3111
Epoch 6/10
154/154 ━━━━━━━━━━━━━━━━━━━━ 5s 32ms/step - accuracy: 0.9914 - loss: 0.0298 - val_accuracy: 0.9069 - val_loss: 0.3471
Epoch 7/10
154/154 ━━━━━━━━━━━━━━━━━━━━ 5s 30ms/step - accuracy: 0.9785 - loss: 0.0630 - val_accuracy: 0.9082 - val_loss: 0.2861
Epoch 8/10
154/154 ━━━━━━━━━━━━━━━━━━━━ 5s 31ms/step - accuracy: 0.9928 - loss: 0.0283 - val_accu

### Performance Metrics And Accuracy

In [57]:
y_pred=model.predict(X_test)

189/189 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step


In [58]:
y_pred=np.where(y_pred > 0.6, 1,0) ##AUC ROC Curve

In [59]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

array([[3097,  318],
       [ 241, 2378]])

In [60]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9073583029499502

In [61]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.93      0.91      0.92      3415
           1       0.88      0.91      0.89      2619

    accuracy                           0.91      6034
   macro avg       0.90      0.91      0.91      6034
weighted avg       0.91      0.91      0.91      6034



In [64]:
import pickle 
with open("vectorizer.pkl", "wb") as f:
    pickle.dump(onehot_repr , f)


with open("model.pkl", "wb") as f:
    pickle.dump(model, f)


In [65]:
model.save("model.h5")      